```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, explode



from sedona.spark import *
from utilities import getConfig

## Setup Sedona environment

In [2]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.5.0,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")

23/10/18 22:38:41 WARN Utils: Your hostname, Nileshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.24.19.124 instead (on interface en0)
23/10/18 22:38:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/nileshgajwani/.ivy2/cache
The jars for the packages stored in: /Users/nileshgajwani/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5d1ede8b-02f5-421d-a31c-93ed390d8872;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/nileshgajwani/Downloads/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.5.0 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 75ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.5.0 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-5d1ede8b-02f5-421d-a31c-93ed390d8872
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/2ms)
23/10/18 22:38:41 WARN NativeCodeLoader: Unable to load

## Read countries shapefile into a Sedona DataFrame 
Data link: https://www.naturalearthdata.com/downloads/50m-cultural-vectors/

In [3]:
countries = ShapefileReader.readToGeometryRDD(sc, "data/ne_50m_admin_0_countries_lakes/")
countries_df = Adapter.toDf(countries, sedona)
countries_df.createOrReplaceTempView("country")
countries_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- LABELRANK: string (nullable = true)
 |-- SOVEREIGNT: string (nullable = true)
 |-- SOV_A3: string (nullable = true)
 |-- ADM0_DIF: string (nullable = true)
 |-- LEVEL: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- ADMIN: string (nullable = true)
 |-- ADM0_A3: string (nullable = true)
 |-- GEOU_DIF: string (nullable = true)
 |-- GEOUNIT: string (nullable = true)
 |-- GU_A3: string (nullable = true)
 |-- SU_DIF: string (nullable = true)
 |-- SUBUNIT: string (nullable = true)
 |-- SU_A3: string (nullable = true)
 |-- BRK_DIFF: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NAME_LONG: string (nullable = true)
 |-- BRK_A3: string (nullable = true)
 |-- BRK_NAME: string (nullable = true)
 |-- BRK_GROUP: string (nullable = true)
 |-- ABBREV: string (nullable = true)
 |-- POSTAL: string (nullable = true)
 |-- FORMAL_EN: st

23/10/18 22:38:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Read airports shapefile into a Sedona DataFrame 
Data link: https://www.naturalearthdata.com/downloads/50m-cultural-vectors/

In [4]:
airports = ShapefileReader.readToGeometryRDD(sc, "data/ne_50m_airports/")
airports_df = Adapter.toDf(airports, sedona)
airports_df.createOrReplaceTempView("airport")
airports_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)



## Run Spatial Join using SQL API

In [5]:
result = sedona.sql("SELECT c.geometry as country_geom, c.NAME_EN, a.geometry as airport_geom, a.name FROM country c, airport a WHERE ST_Contains(c.geometry, a.geometry)")

## Run Spatial Join using RDD API

In [6]:
airports_rdd = Adapter.toSpatialRdd(airports_df, "geometry")
# Drop the duplicate name column in countries_df
countries_df = countries_df.drop("NAME")
countries_rdd = Adapter.toSpatialRdd(countries_df, "geometry")

airports_rdd.analyze()
countries_rdd.analyze()

# 4 is the num partitions used in spatial partitioning. This is an optional parameter
airports_rdd.spatialPartitioning(GridType.KDBTREE, 4)
countries_rdd.spatialPartitioning(airports_rdd.getPartitioner())

buildOnSpatialPartitionedRDD = True
usingIndex = True
considerBoundaryIntersection = True
airports_rdd.buildIndex(IndexType.QUADTREE, buildOnSpatialPartitionedRDD)

result_pair_rdd = JoinQueryRaw.SpatialJoinQueryFlat(airports_rdd, countries_rdd, usingIndex, considerBoundaryIntersection)

result2 = Adapter.toDf(result_pair_rdd, countries_rdd.fieldNames, airports.fieldNames, sedona)

result2.createOrReplaceTempView("join_result_with_all_cols")
# Select the columns needed in the join
result2 = sedona.sql("SELECT leftgeometry as country_geom, NAME_EN, rightgeometry as airport_geom, name FROM join_result_with_all_cols")

[('3.0', '2.12', '1.5.0')]


## Print spatial join results

In [7]:
# The result of SQL API
result.show()
# The result of RDD API
result2.show()

23/10/18 22:38:46 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+--------------------+--------------------+--------------------+--------------------+
|        country_geom|             NAME_EN|        airport_geom|                name|
+--------------------+--------------------+--------------------+--------------------+
|MULTIPOLYGON (((1...|Taiwan           ...|POINT (121.231370...|Taoyuan          ...|
|MULTIPOLYGON (((5...|Netherlands      ...|POINT (4.76437693...|Schiphol         ...|
|POLYGON ((103.969...|Singapore        ...|POINT (103.986413...|Singapore Changi ...|
|MULTIPOLYGON (((-...|United Kingdom   ...|POINT (-0.4531566...|London Heathrow  ...|
|MULTIPOLYGON (((-...|United States of ...|POINT (-149.98172...|Anchorage Int'l  ...|
|MULTIPOLYGON (((-...|United States of ...|POINT (-84.425397...|Hartsfield-Jackso...|
|MULTIPOLYGON (((1...|People's Republic...|POINT (116.588174...|Beijing Capital  ...|
|MULTIPOLYGON (((-...|Colombia         ...|POINT (-74.143371...|Eldorado Int'l   ...|
|MULTIPOLYGON (((6...|India            ...|POINT (72.8

## Group airports by country

In [8]:
# result.createOrReplaceTempView("result")
result2.createOrReplaceTempView("result")
groupedresult = sedona.sql("SELECT c.NAME_EN, c.country_geom, count(*) as AirportCount FROM result c GROUP BY c.NAME_EN, c.country_geom")
groupedresult.show()
groupedresult.createOrReplaceTempView("grouped_result")

+--------------------+--------------------+------------+
|             NAME_EN|        country_geom|AirportCount|
+--------------------+--------------------+------------+
|Cuba             ...|MULTIPOLYGON (((-...|           1|
|Mexico           ...|MULTIPOLYGON (((-...|          12|
|Panama           ...|MULTIPOLYGON (((-...|           1|
|Nicaragua        ...|POLYGON ((-83.157...|           1|
|Honduras         ...|MULTIPOLYGON (((-...|           1|
|Colombia         ...|MULTIPOLYGON (((-...|           4|
|United States of ...|MULTIPOLYGON (((-...|          35|
|Ecuador          ...|MULTIPOLYGON (((-...|           1|
|The Bahamas      ...|MULTIPOLYGON (((-...|           1|
|Peru             ...|POLYGON ((-69.965...|           1|
|Guatemala        ...|POLYGON ((-92.235...|           1|
|Canada           ...|MULTIPOLYGON (((-...|          15|
|Venezuela        ...|MULTIPOLYGON (((-...|           3|
|Argentina        ...|MULTIPOLYGON (((-...|           3|
|Bolivia          ...|MULTIPOLY

## Visualize the number of airports in each country

In [9]:
# sedona_kepler_map = SedonaKepler.create_map(df=groupedresult, name="AirportCount", config=getConfig())
# sedona_kepler_map

In [10]:
h3_df = sedona.sql("SELECT g.NAME_EN, g.country_geom, ST_S2CellIDs(g.country_geom, 3) as s2_cellID from grouped_result g")#groupedresult.selectExpr("ST_H3CellIDs(country_geom, 3, true) as h3_cellId")
h3_df.show(1)
h3_df.printSchema()
h3_df.createOrReplaceTempView("grouped_s2")

+--------------------+--------------------+--------------------+
|             NAME_EN|        country_geom|           s2_cellID|
+--------------------+--------------------+--------------------+
|Cuba             ...|MULTIPOLYGON (((-...|[-859286808902290...|
+--------------------+--------------------+--------------------+
only showing top 1 row

root
 |-- NAME_EN: string (nullable = true)
 |-- country_geom: geometry (nullable = true)
 |-- s2_cellID: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [11]:
exploded_df = h3_df.select(h3_df.NAME_EN, h3_df.country_geom, explode(h3_df.s2_cellID).alias("exploded_cellIds"))
exploded_df.printSchema()
exploded_df.show(2)
exploded_df.createOrReplaceTempView("exploded_cells")

root
 |-- NAME_EN: string (nullable = true)
 |-- country_geom: geometry (nullable = true)
 |-- exploded_cellIds: long (nullable = true)

+--------------------+--------------------+--------------------+
|             NAME_EN|        country_geom|    exploded_cellIds|
+--------------------+--------------------+--------------------+
|Cuba             ...|MULTIPOLYGON (((-...|-8592868089022906368|
|Cuba             ...|MULTIPOLYGON (((-...|-8556839292003942400|
+--------------------+--------------------+--------------------+
only showing top 2 rows



In [12]:
df_s2 = sedona.sql("SELECT s.NAME_EN, s.country_geom, exploded_cellIds as s2 FROM (grouped_s2 s CROSS JOIN exploded_cells e)")
df_s2.show(2)
df_s2.printSchema()

+--------------------+--------------------+--------------------+
|             NAME_EN|        country_geom|                  s2|
+--------------------+--------------------+--------------------+
|Cuba             ...|MULTIPOLYGON (((-...|-8592868089022906368|
|Mexico           ...|MULTIPOLYGON (((-...|-8592868089022906368|
+--------------------+--------------------+--------------------+
only showing top 2 rows

root
 |-- NAME_EN: string (nullable = true)
 |-- country_geom: geometry (nullable = true)
 |-- s2: long (nullable = true)



In [13]:
sedona_kepler_map_s2 = SedonaKepler.create_map(df=df_s2, name="AirportCount", config=getConfig())
sedona_kepler_map_s2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


23/10/18 22:38:53 ERROR Executor: Exception in task 0.0 in stage 58.0 (TID 72)1]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$2719/0x000000080131a840.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at com.esotericsoftware.kryo.io.Output.flush(Output.java:185)
	at com.esotericsoftware.kryo.io.Output.require(Output.java:164)
	at com.esotericsoftware.kryo.io.Output.writeBy

ConnectionRefusedError: [Errno 61] Connection refused